In [34]:
import scipy.stats as st
import numpy as np


# The corrected version of the terachoric correlation is still missing

a = 254       
b = 136
c = 156
d = 193
confidence_level = 0.95


def Binary_2x2_measures(a,b,c,d): 
    sample_size = a+b+c+d
    Corrected_levels_Number = 2 - 1/(sample_size-1)

    # Effect Sizes
    Odds_Ratio = a*d/ (b*c)
    Tetrachoric_Correlation = np.cos(3.14159265359 / (1 + np.sqrt(Odds_Ratio))) 
    Tetrachoric_Basic_Approximation = (Odds_Ratio**0.74 - 1) / (Odds_Ratio**0.74 + 1)
    phi = (a*d - b*c) / np.sqrt((a+b)*(c+d)*(a+c)*(b+d))
    Chambers_R = (((Odds_Ratio + 1)/(Odds_Ratio - 1))  -  ((2*Odds_Ratio * np.log(Odds_Ratio)) / (Odds_Ratio - 1)**2))
    Cramer = phi
    Bias_Corrected_Cramer = np.sqrt((phi**2/(Corrected_levels_Number-1)) ) # CORRECTION IS FROM BERGSMA 2013
    
    # Maximum Corrected Phi (See richard liu 1980)
    max_phi1 = np.sqrt( (  (a+b)/(c+d)) * ((d+b) / (c+a)))
    max_phi2 = np.sqrt( (  (a+c)/(b+d)) * ((a+b) / (c+d)))  
    max_phi = max_phi1 if c + d > b + a else max_phi2
    Max_Corrected_Phi = phi/max_phi

    # Significance_of_phi and max corrected phi
    chi_square_phi = phi**2*sample_size
    p_value = st.chi2.sf((abs(chi_square_phi)), 1)
    chi_square_max_corrected_phi = Max_Corrected_Phi**2*sample_size
    p_value_max_Corrected_phi = st.chi2.sf((abs(chi_square_max_corrected_phi)), 1)

    # Variance of phi (from Bishop et al's., book) (approximation for large samples)
    p1_plus = (a +b)/sample_size # Marginal_Probability_Row1
    p2_plus = (c +d)/sample_size # Marginal_Probability_Row2
    pplus_1 = (a +c)/sample_size # Marginal_Probability_Coloumn1
    pplus_2 = (b +d)/sample_size # Marginal_Probability_Coloumn2
    Probabilities_Product = (p1_plus*p2_plus*pplus_1*pplus_2)
    term1 =  phi + 0.5 * phi**3
    term2 = ((p1_plus - p2_plus) * (pplus_1 - pplus_2)) / np.sqrt(Probabilities_Product)
    term3 = 0.75*phi**2*(((p1_plus - p2_plus)**2 / (p1_plus * p2_plus)) + ((pplus_1 - pplus_2)**2 / (pplus_1 * pplus_2)))
    Variance = (1/sample_size) * (1 - phi**2 + term1 * term2 - term3)

    # Corrected Tetrachoric Correlaiton and Its Variance based on the odds ratio variance (Bonnet, 2005)
    
    # Preaper the Correction 
    r1 = (a + b + 1) / (sample_size + 2)
    r2 = (c + d + 1) / (sample_size + 2)
    c1 = (a + b + 1) / (sample_size + 2)
    c2 = (c + d + 1) / (sample_size + 2)
    Minimal_Probability = min(c1, c2, r1, r2)
    Correction = (1 - abs(r1 - c1) / 5 - (0.5 - Minimal_Probability)**2) / 2
    
    Corrected_Odds_Ratio = (a + 0.5)*(b + 0.5)/((c + 0.5)*(d + 0.5))
    Corrected_Tetrachoric_Correlation = np.cos(3.14159/ (1 + Corrected_Odds_Ratio**Correction))

    Standard_Error_Odds_Ratio = np.sqrt(1/a + 1/b + 1/c + 1/d)
    Standard_Error_Odds_Ratio_Corrected = np.sqrt(1 / (a + 0.5) + 1 / (b + 0.5) + 1 / (c + 0.5) + 1 / (d + 0.5))

    K = (3.14159 * 0.5 * Odds_Ratio**0.5) * np.sin(3.14159 / (1 + Odds_Ratio**0.5)) / (1 + Odds_Ratio**0.5)**2
    Corrected_K = (3.14159 * Correction * Corrected_Odds_Ratio**Correction) * np.sin(3.14159/(1 + Corrected_Odds_Ratio**Correction)) / (1 + Corrected_Odds_Ratio**Correction)**2
    
    Standard_Error_Tetrachoric = K * Standard_Error_Odds_Ratio_Corrected
    Standard_Error_Tetrachoric_Corrected = Corrected_K * Standard_Error_Odds_Ratio_Corrected

    #Wallis Swing and percentage Swing Measures - Coloumns Independent
    Wallis_Swing_d_coloumns_indpendnt =  a/(a+b) - c/(c+d)
    Wallis_Perecentage_Swing_coloumns_independent =  Wallis_Swing_d_coloumns_indpendnt /  (a/(a+b))
    Estimate = (a+c)/sample_size
    Standard_Error_coloumns_independent = np.sqrt(( (Estimate*(1-Estimate)) * (1/(a+b)+(1/(c+d)))))
    z_crit = st.norm.ppf(confidence_level + ((1 - confidence_level) / 2))
    Lower_CI_swing_d_Wallis = Wallis_Swing_d_coloumns_indpendnt - Standard_Error_coloumns_independent*z_crit
    Upper_CI_swing_d_Wallis = Wallis_Swing_d_coloumns_indpendnt + Standard_Error_coloumns_independent*z_crit
    Lower_CI_Percentage_swing_d_Wallis = Lower_CI_swing_d_Wallis / (a/(a+b))
    Upper_CI_Percentage_swing_d_Wallis = Upper_CI_swing_d_Wallis / (a/(a+b))

    #Wallis Swing and percentage Swing Measures - Rows Independent
    Wallis_Swing_d_rows_indpendnt =  a/(a+c) - b/(b+d)
    Wallis_Perecentage_Swing_rows_independent =  Wallis_Swing_d_rows_indpendnt /  (a/(a+c))
    Estimate = (a+b)/sample_size
    Standard_Error_rows_independent = np.sqrt(( (Estimate*(1-Estimate)) * (1/(a+c)+(1/(b+d)))))
    z_crit = st.norm.ppf(confidence_level + ((1 - confidence_level) / 2))
    Lower_CI_swing_d_Wallis_rows = Wallis_Swing_d_coloumns_indpendnt - Standard_Error_coloumns_independent*z_crit
    Upper_CI_swing_d_Wallis_rows = Wallis_Swing_d_coloumns_indpendnt + Standard_Error_coloumns_independent*z_crit
    Lower_CI_Percentage_swing_rows_d_Wallis = Lower_CI_swing_d_Wallis_rows / (a/(a+c))
    Upper_CI_Percentage_swing_rows_d_Wallis = Upper_CI_swing_d_Wallis_rows / (a/(a+c))



    results = {}

    results["Odds Ratio"] = round(Odds_Ratio, 4)
    results["Chambers r"] = round(Chambers_R, 4)
    
    # Tetrachoric Correlation
    results["Tetrachoric Correlation"] = round(Tetrachoric_Correlation, 4)
    results["Tetrachoric Approximation"] = round(Tetrachoric_Basic_Approximation, 4)
    results["Tetrachoric Corrected"] = round(Corrected_Tetrachoric_Correlation, 4)
    results["Standard_Error_Tetrachoric"] = round(Standard_Error_Tetrachoric, 4)
    results["Standard_Error_Tetrachoric_Corrected"] = round(Standard_Error_Tetrachoric_Corrected, 4)

    # Phi and max phi
    results["Phi / Cohen's w"] = round(phi, 4)
    results["Phi chi square statistic"] = round(chi_square_phi, 4)
    results["Phi p_value"] = round(p_value, 4)
    results["maxphi"] = round(max_phi, 4)
    results["Max_Corrected_Phi (Liu, 1980)"] = round(Max_Corrected_Phi, 4)
    results["Phi max_Corrected_p_value"] = round(p_value_max_Corrected_phi, 4)
    results["Phi Variance"] = round(Variance, 7)
    results["chi square phi"] = round(chi_square_phi, 5)
    results["chisquare max corrected phi"] = round(chi_square_max_corrected_phi, 5)

    # Cramer V (Which is equal to phi) and it's biased corrected version
    results["Cramer's V"] = round(Cramer, 4)
    results["Bias Corrected Cramer (Bergsma, 2013)"] = round(Bias_Corrected_Cramer, 4) # Bergsma, 2013

    # Wallis Measures of effect size
    results["Wallis Swing D (coloumns independent)"] = round(Wallis_Swing_d_coloumns_indpendnt, 4)
    results["Wallis Swing D Standard Error"] = round(Standard_Error_coloumns_independent, 5)
    results["Wallis Percentage Swing (coloumns independent)"] = round(Wallis_Perecentage_Swing_coloumns_independent, 4)
    results["CI_swing_d_Wallis"] = f"({round(Lower_CI_swing_d_Wallis, 4)}, {round(Upper_CI_swing_d_Wallis, 4)})"
    results["CI_Percentage_swing_d_Wallis"] = f"({round(Lower_CI_Percentage_swing_d_Wallis, 4)}, {round(Upper_CI_Percentage_swing_d_Wallis, 4)})"

    results["Wallis Swing D (rows independent)"] = round(Wallis_Swing_d_rows_indpendnt, 4)
    results["Wallis Swing D Standard Error (rows)"] = round(Standard_Error_rows_independent, 5)
    results["Wallis Percentage Swing (rows independent)"] = round(Wallis_Perecentage_Swing_rows_independent, 4)
    results["CI_swing_d_Wallis (rows)"] = f"({round(Lower_CI_swing_d_Wallis_rows, 4)}, {round(Upper_CI_swing_d_Wallis_rows, 4)})"
    results["CI_Percentage_swing_d_Wallis (rows)"] = f"({round(Lower_CI_Percentage_swing_rows_d_Wallis, 4)}, {round(Upper_CI_Percentage_swing_rows_d_Wallis, 4)})"

    #result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    #return result_str

    return results


Binary_2x2_measures(a,b,c,d)






{'Odds Ratio': 2.3106,
 'Chambers r': 0.2728,
 'Tetrachoric Correlation': 0.3185,
 'Tetrachoric Approximation': 0.3003,
 'Tetrachoric Corrected': 0.0538,
 'Standard_Error_Tetrachoric': 0.0538,
 'Standard_Error_Tetrachoric_Corrected': 0.0591,
 'Phi': 0.2052,
 'Phi chi square statistic': 31.1209,
 'Phi p_value': 0.0,
 'maxphi': 1.1801,
 'Max_Corrected_Phi)': 0.1739,
 'Phi max_Corrected_p_value': 0.0,
 'Phi Variance': 0.0013005,
 'chi square phi': 31.12087,
 'chi square max corrected phi': 22.34727,
 "Cramer's V)": 0.2052,
 'Bias Corrected Cramer)': 0.2054,
 'Wallis Swing D (coloumns independent)': 0.2043,
 'Wallis Swing D Standard Error': 0.03662,
 'Wallis Percentage Swing (coloumns independent)': 0.3137,
 'CI_swing_d_Wallis': '(0.1325, 0.2761)',
 'CI_Percentage_swing_d_Wallis': '(0.2035, 0.4239)',
 'Wallis Swing D (rows independent)': 0.2061,
 'Wallis Swing D Standard Error (rows)': 0.03695,
 'Wallis Percentage Swing (rows independent)': 0.3327,
 'CI_swing_d_Wallis (rows)': '(0.1325, 0.